In [4]:
import tensorflow as tf

In [5]:
padding = "SAME"  #@param ['SAME', 'VALID' ]
num_output_classes = 29  #@param {type: "number"}
batch_size = 32  #@param {type: "number"}
learning_rate = 0.001  #@param {type: "number"}

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define paths to train, test, and validation directories
train_path = 'C:/Users/Dell/Downloads/GSASLdatasets/train/'
val_path = 'C:/Users/Dell/Downloads/GSASLdatasets/val/'
test_path = 'C:/Users/Dell/Downloads/GSASLdatasets/test/'

In [7]:
img_height, img_width = 100, 100

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_data = train_datagen.flow_from_directory(directory=train_path,
                                         target_size=(img_height, img_width),
                                         batch_size=batch_size,
                                          color_mode='grayscale',
                                         class_mode='categorical')

val_data = val_datagen.flow_from_directory(directory=val_path,
                                       target_size=(img_height, img_width),
                                       batch_size=batch_size,
                                        color_mode='grayscale',
                                       class_mode='categorical')

test_data = test_datagen.flow_from_directory(directory=test_path,
                                        target_size=(img_height, img_width),
                                        batch_size=batch_size,
                                         color_mode='grayscale',
                                        class_mode='categorical',
                                        shuffle=False)

Found 69600 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [24]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_path,
    labels="inferred",  # Automatically infer labels from directory structure
    label_mode="categorical",  # Use categorical labels
    color_mode="grayscale",  # Set the color mode
    batch_size=batch_size,  # Set batch size
    image_size=(img_height, img_width),  # Set image size
    shuffle=True,  # Shuffle the dataset
)


Found 69600 files belonging to 29 classes.


In [11]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_data,
    output_types=(tf.float32, tf.float32),  # Adjust the data types as per your dataset
    output_shapes=([None, img_height, img_width, 1], [None, 29])
)

# Convert val_data to tf.data.Dataset
val_dataset = tf.data.Dataset.from_generator(
    lambda: val_data,
    output_types=(tf.float32, tf.float32),  # Adjust the data types as per your dataset
    output_shapes=([None, img_height, img_width, 1], [None, 29])
)

# Optionally shuffle and batch the datasets
train_dataset = train_dataset.shuffle(1024).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [25]:

leaky_relu_alpha = 0.2 #@param {type: "number"}
dropout_rate = 0.5 #@param {type: "number"}

def conv2d( inputs , filters , stride_size ):
    out = tf.nn.conv2d( inputs , filters , strides=[ 1 , stride_size , stride_size , 1 ] , padding=padding ) 
    return tf.nn.leaky_relu( out , alpha=leaky_relu_alpha ) 

def maxpool( inputs , pool_size , stride_size ):
    return tf.nn.max_pool2d( inputs , ksize=[ 1 , pool_size , pool_size , 1 ] , padding='VALID' , strides=[ 1 , stride_size , stride_size , 1 ] )

def dense( inputs , weights ):
    x = tf.nn.leaky_relu( tf.matmul( inputs , weights ) , alpha=leaky_relu_alpha )
    return tf.nn.dropout( x , rate=dropout_rate )

In [37]:
output_classes = 29
initializer = tf.initializers.glorot_uniform()
def get_weight( shape , name ):
    return tf.Variable( initializer( shape ) , name=name , trainable=True , dtype=tf.float32 )

shapes = [
    [ 3 , 3 , 1 , 16 ] , 
    [ 3 , 3 , 16 , 16 ] , 
    [ 3 , 3 , 16 , 32 ] , 
    [ 3 , 3 , 32 , 32 ] ,
    [ 3 , 3 , 32 , 64 ] , 
    [ 3 , 3 , 64 , 64 ] ,
    [ 3 , 3 , 64 , 128 ] , 
    [ 3 , 3 , 128 , 128 ] ,
    [ 3 , 3 , 128 , 256 ] , 
    [ 3 , 3 , 256 , 256 ] ,
    [ 3 , 3 , 256 , 512 ] , 
    [ 3 , 3 , 512 , 512 ] ,
    [ 8192 , 3600 ] , 
    [ 3600 , 2400 ] ,
    [ 2400 , 1600 ] , 
    [ 1600 , 800 ] ,
    [ 800 , 64 ] ,
    [ 64 , output_classes ] ,
]

weights = []
for i in range( len( shapes ) ):
    weights.append( get_weight( shapes[ i ] , 'weight{}'.format( i ) ) )

In [38]:
def model( x ) :
    x = tf.cast( x , dtype=tf.float32 )
    c1 = conv2d( x , weights[ 0 ] , stride_size=1 ) 
    c1 = conv2d( c1 , weights[ 1 ] , stride_size=1 ) 
    p1 = maxpool( c1 , pool_size=2 , stride_size=2 )
    
    c2 = conv2d( p1 , weights[ 2 ] , stride_size=1 )
    c2 = conv2d( c2 , weights[ 3 ] , stride_size=1 ) 
    p2 = maxpool( c2 , pool_size=2 , stride_size=2 )
    
    c3 = conv2d( p2 , weights[ 4 ] , stride_size=1 ) 
    c3 = conv2d( c3 , weights[ 5 ] , stride_size=1 ) 
    p3 = maxpool( c3 , pool_size=2 , stride_size=2 )
    
    c4 = conv2d( p3 , weights[ 6 ] , stride_size=1 )
    c4 = conv2d( c4 , weights[ 7 ] , stride_size=1 )
    p4 = maxpool( c4 , pool_size=2 , stride_size=2 )

    c5 = conv2d( p4 , weights[ 8 ] , stride_size=1 )
    c5 = conv2d( c5 , weights[ 9 ] , stride_size=1 )
    p5 = maxpool( c5 , pool_size=2 , stride_size=2 )

    c6 = conv2d( p5 , weights[ 10 ] , stride_size=1 )
    c6 = conv2d( c6 , weights[ 11 ] , stride_size=1 )
    p6 = maxpool( c6 , pool_size=2 , stride_size=2 )

    flatten = tf.reshape( p6 , shape=( tf.shape( p6 )[0] , -1 ))

    d1 = dense( flatten , weights[ 12 ] )
    d2 = dense( d1 , weights[ 13 ] )
    d3 = dense( d2 , weights[ 14 ] )
    d4 = dense( d3 , weights[ 15 ] )
    d5 = dense( d4 , weights[ 16 ] )
    logits = tf.matmul( d5 , weights[ 17 ] )

    return tf.nn.softmax( logits )

In [46]:
from tensorflow.keras.models import Model
input_shape = (100 ,100, 1)  # Define your input shape
inputs = tf.keras.Input(shape=input_shape)
outputs = model(inputs)
keras_model = Model(inputs=inputs, outputs=outputs)

ValueError: Exception encountered when calling layer "tf.nn.conv2d" (type TFOpLambda).

Depth of input (1) is not a multiple of input depth of filter (3) for '{{node tf.nn.conv2d/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, tf.nn.conv2d/Conv2D/ReadVariableOp)' with input shapes: [?,100,100,1], [3,3,3,16].

Call arguments received by layer "tf.nn.conv2d" (type TFOpLambda):
  • input=tf.Tensor(shape=(None, 100, 100, 1), dtype=float32)
  • filters=<tf.Variable 'weight0:0' shape=(3, 3, 3, 16) dtype=float32>
  • strides=['1', '1', '1', '1']
  • padding='SAME'
  • data_format=NHWC
  • dilations=None
  • name=None

In [39]:
def loss( pred , target ):
    return tf.losses.categorical_crossentropy( target , pred )

optimizer = tf.optimizers.Adam( learning_rate )

# def train_step( model, inputs , outputs ):
#     with tf.GradientTape() as tape:
#         current_loss = loss( model( inputs ), outputs)
#     grads = tape.gradient( current_loss , weights )
#     optimizer.apply_gradients( zip( grads , weights ) )
#     print( tf.reduce_mean( current_loss ) )
# Define your train_step function
def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = tf.keras.losses.categorical_crossentropy(labels, predictions)
        total_loss = tf.reduce_mean(loss)

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [40]:
# optimizer = tf.keras.optimizers.Adam()
# loss_function = tf.keras.losses.CategoricalCrossentropy()

# # Define a function for the training step
# @tf.function
# def train_step(images, labels, model, optimizer, loss_function):
#     with tf.GradientTape() as tape:
#         predictions = model(images)
#         loss = loss_function(labels, predictions)
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     return loss

# # Set the number of epochs
# num_epochs = 10

# # Training loop
# for epoch in range(num_epochs):
#     epoch_loss = 0.0
#     for images, labels in train_dataset:  # Assuming you have your training dataset
#         images = tf.cast(images, tf.float32)  # Cast images to float32
#         labels = tf.cast(labels, tf.float32)  # Cast labels to float32
#         loss = train_step(images, labels, model, optimizer, loss_function)
#         epoch_loss += loss
#     average_loss = epoch_loss / len(train_dataset)  # Calculate average epoch loss
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}')

ValueError: in user code:

    File "C:\Users\Dell\AppData\Local\Temp\ipykernel_16228\3881182609.py", line 8, in train_step  *
        predictions = model(images)
    File "C:\Users\Dell\AppData\Local\Temp\ipykernel_16228\814714352.py", line 3, in model  *
        c1 = conv2d( x , weights[ 0 ] , stride_size=1 )
    File "C:\Users\Dell\AppData\Local\Temp\ipykernel_16228\1590981331.py", line 5, in conv2d  *
        out = tf.nn.conv2d( inputs , filters , strides=[ 1 , stride_size , stride_size , 1 ] , padding=padding )

    ValueError: Depth of input (1) is not a multiple of input depth of filter (3) for '{{node Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](images, Conv2D/ReadVariableOp)' with input shapes: [32,100,100,1], [3,3,3,16].


In [42]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)


In [45]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.utils import plot_model

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, restore_best_weights=True, verbose=1)
reduce_learning_rate = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.6, verbose=1)

history = tf.keras.fit(train_data,
                    validation_data=val_data,
                    epochs=15,
                    callbacks=[early_stopping, reduce_learning_rate],
                    verbose=1)


AttributeError: module 'tensorflow.keras' has no attribute 'fit'